In [2]:
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import xarray as xr
import pandas as pd
import scipy
from datetime import datetime, timedelta
import time
import seaborn
import matplotlib.dates as mdates
import bottleneck as bn
import PyCO2SYS as pyco2
import gsw
import math
import netCDF4 as nc
import requests

from importlib import reload
import warnings
warnings.filterwarnings('ignore')



# Set up API credentials

API credentials are in a different folder so I have to change the path

In [3]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
#Name of our Service Account Key
google_key_file = '../phyto/servicecredentials.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

# URL retrieval

In [65]:
datalist = ['ALAWAI_w_meta.txt',
            'BOBOA_w_meta.txt',
            'BTM_w_meta.txt',
            'CAPEARAGO_w_meta.txt',
            'CAPEELIZABETH_w_meta.txt',
            'CCE1_w_meta.txt',
            'CCE2_w_meta.txt',
            'CHABA_w_meta.txt',
            'CHEECAROCKS_w_meta.txt',
            'CHUUK_w_meta.txt',
            'COASTALLA_w_meta.txt',
            'COASTALMS_w_meta.txt',
            'CRESCENTREEF_w_meta.txt',
            'CRIMP1_w_meta.txt',
            'CRIMP2_w_meta.txt',
            'DABOB_w_meta.txt',
            'FIRSTLANDING_w_meta.txt',
            'GAKOA_w_meta.txt',
            'GRAYSREEF_w_meta.txt',
            'GULFOFMAINE_w_meta.txt',
            'HOGREEF_w_meta.txt',
            'ICELAND_w_meta.txt',
            'JKEO_w_meta.txt',
            'KANEOHE_w_meta.txt',
            'KEO_w_meta.txt',
            'KILONALU_w_meta.txt',
            'KODIAK_w_meta.txt',
            'LAPARGUERA_w_meta.txt',
            'M2_w_meta.txt',
            'NH10_w_meta.txt',
            'PAPA_w_meta.txt',
            'SEAK_w_meta.txt',
            'SOFS_w_meta.txt',
            'STRATUS_w_meta.txt',
            'TAO110W_w_meta.txt',
            'TAO125W_w_meta.txt',
            'TAO140W_w_meta.txt',
            'TAO155W_w_meta.txt',
            'TAO165E_w_meta.txt',
            'TAO170W_w_meta.txt',
            'TAO8S165E_w_meta.txt',
            'TWANOH_w_meta.txt',
            'WHOTS_w_meta.txt']

In [66]:
urlbase = 'https://data.pmel.noaa.gov/pmel/erddap/files/all_pmel_co2_moorings/'


In [68]:
for i in datalist:
    if i == 'ALAWAI_w_meta.txt':
        URL = urlbase + i
        df = pd.read_csv(URL, skiprows=115)
    else:
        URL = urlbase + i 
        df2 = pd.read_csv(URL, skiprows=115)
        df = pd.concat([df, df2])
    # print(i)

# Drop NaNs and reindex

In [111]:
df0 = df.dropna(how = 'any')
dff = df0.reset_index(drop=True)
dff

,station_id,longitude,latitude,time,SST,SSS,pCO2_sw,pCO2_air,xCO2_air,pH_sw,DOXY,CHL,NTU
0,BOBOA,90.00,15.00,2014-12-07 15:17,28.490,32.488,383.5,381.6,397.2,8.036,186.7,0.031,0.068
1,BOBOA,90.00,15.00,2014-12-08 15:17,28.632,32.722,379.8,382.5,397.8,8.038,187.0,0.038,0.075
2,BOBOA,90.00,15.00,2014-12-08 18:17,28.615,32.767,381.6,381.9,397.3,8.040,186.5,0.031,0.070
3,BOBOA,90.00,15.00,2014-12-09 15:17,28.528,32.469,373.2,389.7,404.8,8.053,188.3,0.035,0.069
4,BOBOA,90.00,15.00,2014-12-09 18:17,28.467,32.529,376.0,387.9,403.2,8.056,188.2,0.020,0.063
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10619,whots,-157.98,22.67,2019-05-12 09:17,25.180,34.535,395.4,399.4,410.4,8.061,212.2,0.127,0.126
10620,whots,-157.98,22.67,2019-05-12 12:17,25.119,34.527,392.1,399.0,410.5,8.060,212.2,0.118,0.171
10621,whots,-157.98,22.67,2019-05-13 09:17,25.476,34.566,392.7,400.2,411.0,8.061,210.9,0.122,0.213
10622,whots,-157.98,22.67,2019-05-13 12:17,25.477,34.573,397.3,399.5,410.8,8.054,211.1,0.119,0.044


# Round coordinates to match model coordinates

In [101]:
for i in dff.index:
    lati = dff['latitude'][i]
    roundedlat = round(lati)
    if roundedlat > lati:
        finallat = roundedlat - 0.5
    else:
        finallat = roundedlat + 0.5
    longi = dff['longitude'][i]
    roundedlon = round(longi)
    if roundedlon > longi:
        finallon = roundedlon - 0.5
    else:
        finallon = roundedlon + 0.5
    dff['latitude'][i] = finallat
    dff['longitude'][i] = finallon

In [107]:
dff

,station_id,longitude,latitude,time,SST,SSS,pCO2_sw,pCO2_air,xCO2_air,pH_sw,DOXY,CHL,NTU
0,BOBOA,90.5,15.5,2014-12-07 15:17,28.490,32.488,383.5,381.6,397.2,8.036,186.7,0.031,0.068
1,BOBOA,90.5,15.5,2014-12-08 15:17,28.632,32.722,379.8,382.5,397.8,8.038,187.0,0.038,0.075
2,BOBOA,90.5,15.5,2014-12-08 18:17,28.615,32.767,381.6,381.9,397.3,8.040,186.5,0.031,0.070
3,BOBOA,90.5,15.5,2014-12-09 15:17,28.528,32.469,373.2,389.7,404.8,8.053,188.3,0.035,0.069
4,BOBOA,90.5,15.5,2014-12-09 18:17,28.467,32.529,376.0,387.9,403.2,8.056,188.2,0.020,0.063
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10619,whots,-157.5,22.5,2019-05-12 09:17,25.180,34.535,395.4,399.4,410.4,8.061,212.2,0.127,0.126
10620,whots,-157.5,22.5,2019-05-12 12:17,25.119,34.527,392.1,399.0,410.5,8.060,212.2,0.118,0.171
10621,whots,-157.5,22.5,2019-05-13 09:17,25.476,34.566,392.7,400.2,411.0,8.061,210.9,0.122,0.213
10622,whots,-157.5,22.5,2019-05-13 12:17,25.477,34.573,397.3,399.5,410.8,8.054,211.1,0.119,0.044


# Write to spreadsheet shared to Wang Lab Drive

In [109]:
#This is the Worksheet ID
spreadsheet_key = '1jfjQkobvAVB4eIjg6sK3lKis1l-_czaeoK7tGZfg8VE'
#This is the sheet name
wks_name = 'Sheet1'
#We upload the tips data to our Google Sheet. Setting the row_names to False if you did not want the index to be included
d2g.upload(dff, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'Sheet1' id:0>

# Test pull from spreadsheet

In [5]:
#This is the Worksheet ID
spreadsheet_key = '1jfjQkobvAVB4eIjg6sK3lKis1l-_czaeoK7tGZfg8VE'
#This is the sheet name
wks_name = 'Sheet1'

#Opening the worksheet by using Worksheet ID
workbook = gc.open_by_key(spreadsheet_key)
#Selecting which sheet to pulling the data
sheet = workbook.worksheet(wks_name)

#Pulling the data and transform it to the data frame
values = sheet.get_all_values()
df = pd.DataFrame(values[1:], columns = values[0])
df

,station_id,longitude,latitude,time,SST,SSS,pCO2_sw,pCO2_air,xCO2_air,pH_sw,DOXY,CHL,NTU
0,BOBOA,90.5,15.5,2014-12-07 15:17,28.49,32.488,383.5,381.6,397.2,8.036,186.7,0.031,0.068
1,BOBOA,90.5,15.5,2014-12-08 15:17,28.632,32.722,379.8,382.5,397.8,8.038,187.0,0.038,0.075
2,BOBOA,90.5,15.5,2014-12-08 18:17,28.615,32.767,381.6,381.9,397.3,8.04,186.5,0.031,0.07
3,BOBOA,90.5,15.5,2014-12-09 15:17,28.528,32.469,373.2,389.7,404.8,8.053,188.3,0.035,0.069
4,BOBOA,90.5,15.5,2014-12-09 18:17,28.467,32.529,376.0,387.9,403.2,8.056,188.2,0.02,0.063
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10619,whots,-157.5,22.5,2019-05-12 09:17,25.18,34.535,395.4,399.4,410.4,8.061,212.2,0.127,0.126
10620,whots,-157.5,22.5,2019-05-12 12:17,25.119,34.527,392.1,399.0,410.5,8.06,212.2,0.118,0.171
10621,whots,-157.5,22.5,2019-05-13 09:17,25.476,34.566,392.7,400.2,411.0,8.061,210.9,0.122,0.213
10622,whots,-157.5,22.5,2019-05-13 12:17,25.477,34.573,397.3,399.5,410.8,8.054,211.1,0.119,0.044


# Solve carb system for buoys
INPUTS
> `pCO2_sw`: partial pressure of CO2 in seawater [uatm] -- type `4`

> `pH_sw`: seawater pH -- type `3`

> `SSS`: sea surface salinity

> `SST`: sea surface temperature [degC]

In [6]:
results = pyco2.sys(par1=df['pCO2_sw'],par2=df['pH_sw'],par1_type=4,par2_type=3,salinity = df['SSS'], temperature = df['SST'])



## Add fCO2 and Omega to the data

In [11]:
df['fCO2'] = results['fCO2']

In [12]:
df['OmegaAr'] = results['saturation_aragonite']

In [13]:
%run buoydata_retrieval.py